<div align='center'>

# Practica 2

<img src='https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExbXVoOXh6bGtwend4cW5majMwZHRiYzh6b2loZGNseGZmajMxbmhsNiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/aMwdfGSyeYjUKY6vjf/giphy.gif'>

</div>

---

## Ejercicio 1)
¿En el dataset del ejercicio 1 de la práctica 1 indique para cada Job, si se vería beneficiado por una función combiner?

En caso afirmativo,

¿cuál es la implementación de dicha función? ¿Qué datos recibe cada reduce, al utilizar la función combiner?

---

## Ejercicio 2)
Implemente una función combiner para el problema del WordCount.

In [2]:
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "./WordCount/input/"
outputDir = "./WordCount/ejercicio_2/"

def fmap(key, value, context):
    words = value.split()
    for w in words:
        context.write(w, 1)
        
def fred(key, values, context):
    c=0
    for v in values:
        c=c+1
    context.write(key, c)

job = Job(inputDir, outputDir, fmap, fred)
job.setCombiner(fred)
success = job.waitForCompletion()
print(success)

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] ï»¿LA -> 1
[Map] PUERTA -> 1
[Map] DE -> 1
[Map] BRONCE -> 1
[Map] --Â¡Adelante! -> 1
[Map] exclamÃ³ -> 1
[Map] el -> 1
[Map] Prelado, -> 1
[Map] y -> 1
[Map] apareciÃ³ -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: !ah! -> [1]
[Reduce] Clave recibida: "A" -> [1]
[Reduce] Clave recibida: "B" -> [1]
[Reduce] Clave recibida: "Dr. -> [1]
[Reduce] Clave recibida: "El -> [1]
[Reduce] Clave recibida: "purgar", -> [1]
[Reduce] Clave recibida: (aquÃ­ -> [1]
[Reduce] Clave recibida: (con -> [1]
[Reduce] Clave recibida: (naturalmente) -> [1]
[Reduce] Clave recibida: * -> [11, 1, 1, 1, 1]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
!ah!	1
"A"	1
"B"	1
"Dr.	1
"El	1
"purgar",	1
(aquÃ­	1
(con	1
(naturalmente)	1
*	5
... [más resultados omitidos]
True


---

## Ejercicio 3)
Implemente un job MapReduce para calcular el máximo, mínimo, promedio y desvío stándard de las ocurrencias de todas las palabras del dataset Libros.

#### Cuenta ocurrencias de cada palabra.


In [3]:
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "./Libros/input"
wcDir = "./Libros/ej3_wc/"

def fmap_wc(key, value, context):
    # separar por espacios
    for palabra in value.strip().split():
        palabra = palabra.lower().strip(".,;:¡!¿?\"()[]")
        if palabra:
            context.write(palabra, 1)

def fred_wc(key, values, context):
    total = 0
    for v in values:
        total += v
    context.write(key, total)

job_wc = Job(inputDir, wcDir, fmap_wc, fred_wc)
job_wc.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] fray -> 1
[Map] baltasar -> 1
[Map] cuando -> 1
[Map] se -> 1
[Map] hallã³ -> 1
[Map] por -> 1
[Map] fin -> 1
[Map] en -> 1
[Map] la -> 1
[Map] soledad -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: * -> [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, '...']
[Reduce] Clave recibida: --"estimado -> [1]
[Reduce] Clave recibida: --amigo -> [1, 1]
[Reduce] Clave recibida: --angustias -> [1]
[Reduce] Clave recibida: --aquí -> [1]
[Reduce] Clave recibida: --aun -> [1]
[Reduce] Clave recibida: --buenas -> [1, 1]
[Reduce] Clave recibida: --buenos -> [1, 1]
[Reduce] Clave recibida: --comprendo -> [1]
[Reduce] Clave recibida: --doctor -> [1]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
*	15
--"estimado	1
--amigo	2
--angustias	1
--aquí	1
--aun	1
--buenas	2
--buenos	2
--comprendo	1
--doctor	1
... [más resultados omitidos]


True

#### Job de estadísticas (min, max, promedio)

In [13]:
statsDir = "./Libros/ej3_stats/"

def fmap_stats(key, value, context):
    ocurr = int(value.strip())
    context.write("min", ocurr)
    context.write("max", ocurr)
    context.write("avg", ocurr)

def fred_stats(key, values, context):
    if key == "min":
        context.write("min", min(values))
    elif key == "max":
        context.write("max", max(values))
    elif key == "avg":
        total, n = 0, 0
        for v in values:
            total += v
            n += 1
        context.write("promedio", total / n)

job_stats = Job(wcDir, statsDir, fmap_stats, fred_stats)
job_stats.waitForCompletion()


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] min -> 15
[Map] max -> 15
[Map] avg -> 15
[Map] min -> 1
[Map] max -> 1
[Map] avg -> 1
[Map] min -> 1
[Map] max -> 1
[Map] avg -> 1
[Map] min -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: avg -> [15, 1, 1, 1, 1, 1, 1, 1, 1, 3, '...']
[Reduce] Clave recibida: max -> [15, 1, 1, 1, 1, 1, 1, 1, 1, 3, '...']
[Reduce] Clave recibida: min -> [15, 1, 1, 1, 1, 1, 1, 1, 1, 3, '...']
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
promedio	3.4946911196911197
max	839
min	1


True

#### Job de desvío estándar

In [12]:
desvioDir = "./Libros/ej3_desvio/"

import math

def fmap_desvio(key, value, context):
    ocurr = int(value.strip())
    avg = context["avg"]
    context.write("var", (ocurr - avg) ** 2)

def fred_desvio(key, values, context):
    total, n = 0, 0
    for v in values:
        total += v
        n += 1
    # varianza muestral
    var = total / (n - 1)
    context.write("desvio", math.sqrt(var))

# leer promedio del job_stats
with open(statsDir + "output.txt", "r") as f:
    lines = f.readlines()

prom_line = [l for l in lines if l.startswith("promedio")][0]
promedio = float(prom_line.strip().split('\t')[1])

params = {"avg": promedio}
job_desvio = Job(wcDir, desvioDir, fmap_desvio, fred_desvio)
job_desvio.setParams(params)
job_desvio.waitForCompletion()


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] var -> 132.3721324313144
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 6.223483782665732
[Map] var -> 0.24471930390125368
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: var -> [132.3721324313144, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 6.223483782665732, 0.24471930390125368, '...']
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
desvio	22.917022729315015


True

---

## Ejercicio 4
Utilice el dataset Libros para implementar una aplicación MapReduce que devuelva como salida todos los párrafos que tienen una longitud mayor al promedio.


In [6]:
import sys, math
sys.path.append("..")
from MRE import Job

librosDir = "./Libros/input"
out1Dir = "./Libros/ej4_out1/"

# ===== Job 1: promedio de longitud de párrafos =====
def fmap_len(key, value, context):
    length = len(value.strip())
    context.write(1, length)

def fred_len(key, values, context):
    total, n = 0, 0
    for v in values:
        total += v
        n += 1
    avg = total / n if n > 0 else 0
    context.write("promedio", avg)

job1 = Job(librosDir, out1Dir, fmap_len, fred_len)
job1.waitForCompletion()

out2Dir = "./Libros/ej4_out2/"

# ===== Job 2: filtrar párrafos mayores al promedio =====
def fmap_filter(key, value, context):
    length = len(value.strip())
    context.write(value.strip(), length)

def fred_filter(parrafo, values, context):
    avg = context["avg"]
    for v in values:
        if v > avg:
            context.write("Párrafo > promedio", parrafo)

params = {"avg": promedio}
job2 = Job(librosDir, out2Dir, fmap_filter, fred_filter)
job2.setParams(params)
job2.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 1 -> 8
[Map] 1 -> 143
[Map] 1 -> 218
[Map] 1 -> 379
[Map] 1 -> 166
[Map] 1 -> 185
[Map] 1 -> 484
[Map] 1 -> 245
[Map] 1 -> 301
[Map] 1 -> 46
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 1 -> [8, 143, 218, 379, 166, 185, 484, 245, 301, 46, '...']
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
promedio	293.4931506849315
[MRE] INICIANDO ETAPA DE MAPEO...
[Map] EL PAPAGAYO DE HUICHILOBOS -> 26
[Map] Repentinamente oí una exclamación de sorpresa y, volviendo el rostro, me encontré cara a cara con el Padre Montero, mi antiguo condiscípulo, a quien no había visto en cinco años. Fungía de Sacristán mayor de la Catedral y llevaba un manojo de enormes llaves, pues era hora de cerrar el templo, para volver a abrirlo a las tres de la tarde. Inútil me parece relatar el gusto que me dió volver a ver a tan buen amigo mío. Convidóme a almorzar y prometió enseñarme é

True

---

## Ejercicio 5

El dataset website tiene información sobre el tiempo de permanencia de sus usuarios
en cada una de las páginas del sitio. El formato de los datos del dataset es:
<id_user, id_page, time>
Implemente una aplicación MapReduce, utilizando combiners en los casos que
considere necesario, que calcule

> Indique como queda el DAG del proceso completo (las tres consultas)



---

#### a. La página más visitada (la página en la que más tiempo permaneció) para cada usuario

In [7]:
import sys
sys.path.append("..")
from MRE import Job

# directorios de trabajo
inputDir = "./website/input/"
out1 = "./website/out_A1/"
out2 = "./website/out_A2/"

# agrupa por (user, page)
def fmap_1(user_id, value, context):
    parts = value.strip().split('\t')
    page_id, time = parts
    context.write((user_id, page_id), int(time))

# reduce: suma tiempos por (user, page)
def fred_1(key, values, context):
    total_time = 0
    for t in values:
        total_time += t
    context.write(key, total_time)

# agrupa por user
def fmap_2(user_id, value, context):
    parts = value.strip().split('\t')
    page_id, total_time = parts
    context.write(user_id, (page_id, int(total_time)))

# reduce: selecciona la página con más tiempo
def fred_2(user_id, values, context):
    max_time = -1
    best_page = None
    for page_id, t in values:
        if t > max_time:
            max_time = t
            best_page = page_id
    context.write(user_id, best_page)

job1 = Job(inputDir, out1, fmap_1, fred_1)
job1.setCombiner(fred_1)
job1.waitForCompletion()

job2 = Job(out1, out2, fmap_2, fred_2)
job2.waitForCompletion()


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] ('9482', '1590') -> 37
[Map] ('13156', '1084') -> 65
[Map] ('9601', '3819') -> 75
[Map] ('11171', '2899') -> 42
[Map] ('11744', '1272') -> 79
[Map] ('9619', '3160') -> 62
[Map] ('12961', '3803') -> 66
[Map] ('8832', '2613') -> 36
[Map] ('11404', '1016') -> 11
[Map] ('11092', '3363') -> 11
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: ('10000', '2332') -> [81]
[Reduce] Clave recibida: ('10000', '2852') -> [52]
[Reduce] Clave recibida: ('10001', '2903') -> [82]
[Reduce] Clave recibida: ('10002', '1353') -> [27]
[Reduce] Clave recibida: ('10002', '2699') -> [24]
[Reduce] Clave recibida: ('10004', '3676') -> [30]
[Reduce] Clave recibida: ('10005', '3079') -> [21]
[Reduce] Clave recibida: ('10007', '1084') -> [6]
[Reduce] Clave recibida: ('10007', '3095') -> [65]
[Reduce] Clave recibida: ('10008', '1986') -> [6]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

True

---

#### b. El usuario que más páginas distintas visitó

In [8]:
# directorios de salida para inciso B
out3 = "./website/out_B1/"
out4 = "./website/out_B2/"

# cuenta páginas distintas por usuario
def fmap_3(user_id, value, context):
    parts = value.strip().split('\t')
    page_id, _ = parts
    context.write(user_id, page_id)

def fred_3(user_id, values, context):
    distinct_pages = set(values)
    context.write(user_id, len(distinct_pages))

# agrupa todos los (user, count) bajo una misma clave
def fmap_4(user_id, value, context):
    context.write(1, (user_id, int(value.strip())))

def fred_max(_, values, context):
    best_user, max_count = None, -1
    for user, c in values:
        if c > max_count:
            max_count = c
            best_user = user
    context.write("Usuario con más páginas distintas", (best_user, max_count))

job3 = Job(out1, out3, fmap_3, fred_3)
job3.waitForCompletion()

job4 = Job(out3, out4, fmap_4, fred_max)
job4.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 10000 -> 2332
[Map] 10008 -> 2693
[Map] 10024 -> 2491
[Map] 10032 -> 3077
[Map] 10045 -> 2016
[Map] 10060 -> 2797
[Map] 10070 -> 1194
[Map] 10080 -> 1724
[Map] 10092 -> 3902
[Map] 10105 -> 1291
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 10000 -> ['2332', '2852']
[Reduce] Clave recibida: 10001 -> ['2903']
[Reduce] Clave recibida: 10002 -> ['1353', '2699']
[Reduce] Clave recibida: 10004 -> ['3676']
[Reduce] Clave recibida: 10005 -> ['3079']
[Reduce] Clave recibida: 10007 -> ['1084', '3095']
[Reduce] Clave recibida: 10008 -> ['2693', '1986']
[Reduce] Clave recibida: 10009 -> ['1586', '3638', '3926']
[Reduce] Clave recibida: 10011 -> ['2720']
[Reduce] Clave recibida: 10012 -> ['3152']
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
10000	2
10001	1
10002	2
10004	1
10005	1
10007	2
10008	2
10009	3
10011	1
10012	1
... [más r

True

---

#### c. La página más visitada (en cuanto a cantidad de visitas, sin importar el tiempo de permanencia) por todos los usuarios.

In [10]:
# directorios de salida para inciso C
out5 = "./website/out_C1/"
out6 = "./website/out_C2/"
out7 = "./website/out_C3/"

# ---------------- JOB 5 ----------------
# Emite (page_id, 1) por cada visita
def fmap_5(user_id, value, context):
    parts = value.strip().split('\t') 
    page_id, time = parts   # el tiempo no importa acá
    context.write(page_id, 1)

def fred_count(page_id, values, context):
    total_visits = sum(values)
    context.write(page_id, total_visits)

# ---------------- JOB 6 ----------------
# Junta todas las visitas de cada página (ya están como enteros)
def fmap_6(page_id, value, context):
    context.write(page_id, int(value.strip()))

def fred_sum(page_id, values, context):
    total = sum(values)
    context.write(page_id, total)

# ---------------- JOB 7 ----------------
# Busca la página con más visitas
def fmap_7(page_id, value, context):
    context.write(1, (page_id, int(value.strip())))

def fred_max(_, values, context):
    max_count = -1
    best_page = None
    for page, count in values:
        if count > max_count:
            max_count = count
            best_page = page
    context.write("Página más visitada", (best_page, max_count))

job5 = Job(inputDir, out5, fmap_5, fred_count)
job5.waitForCompletion()

job6 = Job(out5, out6, fmap_6, fred_sum)
job6.waitForCompletion()

job7 = Job(out6, out7, fmap_7, fred_max)
job7.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 3876 -> 1
[Map] 3492 -> 1
[Map] 1966 -> 1
[Map] 2419 -> 1
[Map] 1793 -> 1
[Map] 2930 -> 1
[Map] 3415 -> 1
[Map] 2351 -> 1
[Map] 3120 -> 1
[Map] 1675 -> 1
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 1000 -> [1]
[Reduce] Clave recibida: 1001 -> [1, 1, 1]
[Reduce] Clave recibida: 1002 -> [1]
[Reduce] Clave recibida: 1003 -> [1, 1, 1, 1, 1]
[Reduce] Clave recibida: 1005 -> [1, 1]
[Reduce] Clave recibida: 1006 -> [1, 1]
[Reduce] Clave recibida: 1007 -> [1, 1, 1]
[Reduce] Clave recibida: 1008 -> [1]
[Reduce] Clave recibida: 1010 -> [1, 1, 1]
[Reduce] Clave recibida: 1011 -> [1, 1]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
1000	1
1001	3
1002	1
1003	5
1005	2
1006	2
1007	3
1008	1
1010	3
1011	2
... [más resultados omitidos]
[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 1000 -> 1
[Map] 1012 -> 3
[Map] 1025 -> 2
[Map] 1036 -> 1
[

True

---

## 6) Cómo plantearía una solución MapReduce a los siguientes algoritmos secuenciales:

i. entrada

```
textos: array [1..N] of string (dataset libros)
```
ii. algoritmo

```python
a={}; b={}; N = len(textos)
for l in textos:
    words = l.split()
    for w in words:
        a[w] = a[w]+1

for w in a.keys():
    for l in lines:
        words = l.split()
        if w in words:
            b[w]=b[w]+1
for k in a.keys():
    print(k + " = " + str(a[w] * (N / b[w])))
```

El algoritmo secuencial trabaja como una variante de **TF-IDF** (frecuencia de palabra por documento).
Hace tres cosas:

1. Cuenta cuántas veces aparece cada palabra en todo el dataset (`a[w]`).
2. Cuenta en cuántos documentos (líneas o párrafos) aparece cada palabra (`b[w]`).
3. Devuelve para cada palabra: `a[w] * (N / b[w])`, donde `N` es la cantidad total de documentos.

**Cómo se plantea en MapReduce**

Este problema no se resuelve con un solo job, sino con **tres jobs encadenados**:

* **Job 1 (contar ocurrencias totales de palabras):**

  * *Map:* cada mapper toma un documento y por cada palabra emite `(palabra, 1)`.
  * *Reduce:* los reducers suman esos 1 y obtienen cuántas veces aparece cada palabra en todo el dataset.
  * **Salida:** `(palabra, total_ocurrencias)` → esto es `a[w]`.

* **Job 2 (contar en cuántos documentos aparece cada palabra):**

  * *Map:* cada mapper toma un documento y por cada palabra distinta en ese documento emite `(palabra, doc_id)`.
  * *Reduce:* para cada palabra, cuenta la cantidad de `doc_id` distintos → en cuántos documentos aparece.
  * **Salida:** `(palabra, num_docs)` → esto es `b[w]`.

* **Job 3 (cálculo final):**

  * *Map:* se unen las salidas de Job1 y Job2 por clave (la palabra).
  * *Reduce:* con `a[w]` y `b[w]` calcula `a[w] * (N / b[w])`.
  * **Salida final:** `(palabra, score)`.

🔗 El **DAG** se ve así:

```
Dataset Libros
 ├─ Job1 → a[w] = total ocurrencias
 ├─ Job2 → b[w] = nº de documentos con w
 └─ Job3 (join y cálculo) → resultado final
```

---

b.
i. entrada
```
datos: array [1..N] of /<int1, int2, ..., intM> (todos los valores están dentro de un rango de valores conocido, para poder usarlos como índices del tensor)
```
ii. algoritmo

```python
for t in datos:
    v = t.split("\t")
    c = v[-1]
        for a in range(len(v)-1):
        x= v[a]
        m[a][x][c] = m[a][x][c] + 1

max=[[0,0,0], [0,0,0]]
for x in range(len(m)):
    for y in range(len(m[0])):
        for z in range(2):
            if(m[x][y][z] > max[z][0]):
                max[z][0] = m[x][y][z]
                max[z][1]=x
                max[z][2]=y

for z in range(2):
    print(z +";" + max[z][1] +";" + max[z][2])
```



El algoritmo secuencial toma un dataset de tuplas `<x1, x2, …, xM, c>` (atributos + clase).
Hace dos cosas:

1. Construye una especie de “tensor de frecuencias” `m[a][x][c]`, que cuenta cuántas veces el atributo `a` tomó el valor `x` en ejemplos de clase `c`.
2. Busca, para cada clase `c`, cuál es el valor de atributo que más se repite.

**Cómo se plantea en MapReduce**

Este problema se resuelve en **dos jobs**:

* **Job 1 (conteo de triples atributo–valor–clase):**

  * *Map:* para cada tupla `<x1, …, xM, c>`, emite `(a, x, c) → 1` para cada atributo `a` y su valor `x`.
  * *Reduce:* para cada triple `(a, x, c)` suma todas las ocurrencias.
  * **Salida:** `(a, x, c, count)` → cuántas veces apareció el valor `x` del atributo `a` en la clase `c`.

* **Job 2 (buscar máximos por clase):**

  * *Map:* reemite los resultados del Job 1 pero agrupando por clase `c`.
  * *Reduce:* para cada clase `c`, recorre todos los `(a, x, count)` y selecciona el de mayor frecuencia.
  * **Salida final:** `(c, mejor atributo=a, valor=x, count)`.

🔗 El **DAG** se ve así:

```
Dataset Datos
 ├─ Job1 → conteo de (atributo, valor, clase)
 └─ Job2 → selección del máximo por clase
```